<a href="https://colab.research.google.com/github/diogosantanaime/rec_proc/blob/main/run_crf_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U 'scikit-learn<0.24'
! pip install sklearn_crfsuite
! pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=b7206ea5a9606544753ab354365e30922c5f099de8ba0b8619757a04d5e0e3dc
  Stored in directory: /root/.cache/p

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import os
import pickle
import sklearn
import scipy.stats
import sklearn_crfsuite

from nltk import pos_tag, RegexpParser
from nltk.tokenize import word_tokenize
from itertools import chain
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from seqeval.metrics import classification_report, f1_score, accuracy_score
from sklearn_crfsuite import CRF


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def read_dataset_files(dataset_files):
    datasets = {}

    for name, filepath in dataset_files.items():
        with open(filepath, 'r') as file:
            datasets[name] = file.read()

    return datasets

# Dicionário contendo os nomes dos conjuntos de dados e seus respectivos caminhos de arquivo
dataset_files = {
    'train_conll03': '/content/drive/MyDrive/dataset/train_conll03.txt',
    'test_conll03': '/content/drive/MyDrive/dataset/test_conll03.txt',
    'val_conll03': '/content/drive/MyDrive/dataset/val_conll03.txt',
    'train_wikineural': '/content/drive/MyDrive/dataset/train_wikineural.txt',
    'test_wikineural': '/content/drive/MyDrive/dataset/test_wikineural.txt',
    'val_wikineural': '/content/drive/MyDrive/dataset/val_wikineural.txt',
    'train_pet': '/content/drive/MyDrive/dataset/train_pet.txt',
    'test_pet': '/content/drive/MyDrive/dataset/test_pet.txt',
    'val_pet': '/content/drive/MyDrive/dataset/val_pet.txt'
}

# Chama a função para ler os arquivos do conjunto de dados
datasets = read_dataset_files(dataset_files)


In [4]:
# Função para processar os conjuntos de dados
def preprocess_datasets(datasets):
    processed_datasets = {}

    for dataset_name, dataset_content in datasets.items():
        # Substitui espaços em branco por tabulação
        modified_content = dataset_content.replace(' ', '\t')

        # Remove espaços em branco extras e divide em sentenças
        sentences = modified_content.strip().split('\n\n')

        processed_datasets[dataset_name] = sentences

    return processed_datasets

# Chama a função para processar os conjuntos de dados
datasets = preprocess_datasets(datasets)

In [5]:
# Função para remontar os conjunto de dados no formato necessário para o CRF Suite
def remount_data(datasets):
    remounted_datasets = {}
    for dataset, sentences in datasets.items():
        remounted_data = []
        for sentence in sentences:
            remounted_sentence = []
            lines = sentence.strip().split('\n')
            for line in lines:
                word, ner_tag = line.split('\t')
                tokens = word_tokenize(word)
                pos_tags = [tag for _, tag in pos_tag(tokens)]
                for i in range(len(tokens)):
                    remounted_sentence.append((tokens[i], pos_tags[i], ner_tag))
            remounted_data.append(remounted_sentence)
        remounted_datasets[dataset] = remounted_data
    return remounted_datasets

# Chama a função para remontar os datasets no formato crf suite
datasets = remount_data(datasets)

In [6]:
def word2features(sent, i):
    """
    Gera as features para uma palavra específica em uma sentença.

    Argumentos:
        sent (list): A sentença contendo palavras e suas tags.
        i (int): O índice da palavra na sentença.

    Retorna:
        dict: Um dicionário contendo as features da palavra.
    """
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    """
    Gera as features para todas as palavras em uma sentença.

    Argumentos:
        sent (list): A sentença contendo palavras e suas tags.

    Retorna:
        list: Uma lista de dicionários contendo as features de cada palavra.
    """
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    """
    Extrai as labels de uma sentença.

    Argumentos:
        sent (list): A sentença contendo palavras e suas tags.

    Retorna:
        list: Uma lista contendo apenas as labels da sentença.
    """
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    """
    Extrai as palavras de uma sentença.

    Argumentos:
        sent (list): A sentença contendo palavras e suas tags.

    Retorna:
        list: Uma lista contendo apenas as palavras da sentença.
    """
    return [token for token, postag, label in sent]


def sent2features_to_datasets(datasets):
    """
    Converte os conjuntos de dados para o formato necessário para o CRF Suite.

    Argumentos:
        datasets (list): Lista contendo os conjuntos de dados.

    Retorna:
        tuple: Tupla contendo as features (X) e as labels (y) para cada conjunto de dados.
    """
    X = [sent2features(s) for s in datasets]
    y = [sent2labels(s) for s in datasets]
    return X, y


In [9]:
 crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100,
              all_possible_transitions=True)

In [10]:
# Convertendo os conjuntos de dados para o formato necessário para o CRF Suite
X_train, y_train = sent2features_to_datasets(datasets['train_conll03'])
X_test, y_test = sent2features_to_datasets(datasets['test_conll03'])

# Ajustar o modelo CRF aos dados de treinamento
crf.fit(X_train, y_train)

# Realizar a predição do modelo nos dados de teste
y_pred = crf.predict(X_test)

# Obtendo a lista de classes do modelo CRF
labels = list(crf.classes_)

# Removendo a classe 'O' da lista de classes
labels.remove('O')

# Calculando a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calculando o F-score (micro)
f1_micro = f1_score(y_test, y_pred, average='micro')

# Calculando o F-score (macro)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("F-score (micro):", f1_micro)
print("F-score (macro):", f1_macro)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))



F-score (micro): 0.7999291157185895
F-score (macro): 0.7931177214621968
Accuracy: 0.958152649904941
              precision    recall  f1-score   support

         LOC       0.86      0.84      0.85      1724
        MISC       0.79      0.74      0.76       708
         ORG       0.78      0.68      0.72      1684
         PER       0.83      0.85      0.84      1656

   micro avg       0.82      0.78      0.80      5772
   macro avg       0.81      0.78      0.79      5772
weighted avg       0.82      0.78      0.80      5772



In [11]:
# Convertendo os conjuntos de dados para o formato necessário para o CRF Suite
X_train, y_train = sent2features_to_datasets(datasets['train_wikineural'])
X_test, y_test = sent2features_to_datasets(datasets['test_wikineural'])

# Ajustar o modelo CRF aos dados de treinamento
crf.fit(X_train, y_train)

# Realizar a predição do modelo nos dados de teste
y_pred = crf.predict(X_test)

# Obtendo a lista de classes do modelo CRF
labels = list(crf.classes_)

# Removendo a classe 'O' da lista de classes
labels.remove('O')

# Calculando a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calculando o F-score (micro)
f1_micro = f1_score(y_test, y_pred, average='micro')

# Calculando o F-score (macro)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("F-score (micro):", f1_micro)
print("F-score (macro):", f1_macro)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

F-score (micro): 0.7662195224837594
F-score (macro): 0.756895501033221
Accuracy: 0.970442074110611
              precision    recall  f1-score   support

         LOC       0.75      0.84      0.79      1781
        MISC       0.76      0.70      0.72      1538
         ORG       0.80      0.61      0.69      1103
         PER       0.83      0.81      0.82      1601

   micro avg       0.78      0.75      0.77      6023
   macro avg       0.78      0.74      0.76      6023
weighted avg       0.78      0.75      0.76      6023



In [12]:
# Convertendo os conjuntos de dados para o formato necessário para o CRF Suite
X_train, y_train = sent2features_to_datasets(datasets['train_pet'])
X_test, y_test = sent2features_to_datasets(datasets['test_pet'])

# Ajustar o modelo CRF aos dados de treinamento
crf.fit(X_train, y_train)

# Realizar a predição do modelo nos dados de teste
y_pred = crf.predict(X_test)

# Obtendo a lista de classes do modelo CRF
labels = list(crf.classes_)

# Removendo a classe 'O' da lista de classes
labels.remove('O')

# Calculando a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calculando o F-score (micro)
f1_micro = f1_score(y_test, y_pred, average='micro')

# Calculando o F-score (macro)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("F-score (micro):", f1_micro)
print("F-score (macro):", f1_macro)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

F-score (micro): 0.02247191011235955
F-score (macro): 0.01111557818874892
Accuracy: 0.3568627450980392
                        precision    recall  f1-score   support

            ANDGateway       0.00      0.00      0.00         2
              Activity       0.06      0.02      0.03        60
          ActivityData       0.05      0.02      0.02        61
                 Actor       0.05      0.02      0.03        51
ConditionSpecification       0.00      0.00      0.00         7
  FurtherSpecification       0.00      0.00      0.00         4
            XORGateway       0.00      0.00      0.00        10

             micro avg       0.04      0.02      0.02       195
             macro avg       0.02      0.01      0.01       195
          weighted avg       0.04      0.02      0.02       195



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
